In [1]:
import matplotlib.pyplot as plt

from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam, Nadam, RMSprop

Using TensorFlow backend.


## 모델 생성

In [2]:
def price_pred_model(
    window_sizer = 10, save = False, filename = 'model', lstm = 1, dropout_rate = 0.2, optimizers = RMSprop()
    ):
    model = Sequential()
    model.add(
        LSTM(2, return_sequences = True, input_shape = (window_sizer, 2))
    )
    if lstm > 1:
        for steps in range(lstm - 1):
            model.add(
                LSTM(64, return_sequences = True)
            )
            model.add(Dropout(dropout_rate))            
    model.add(
        LSTM(64, return_sequences = False)
    )
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    # model.add(Dense(1, activation = 'linear'))
    model.compile(
        # optimizer = RMSprop(), Adam(), Nadam()
        optimizer = optimizers,
        loss = 'mse')
    model.summary()

    if save == True:
        model.save(f'data/{filename}_unfitted.h5')
        print('모델 저장 완료')
    return model

## 모델 훈련

In [3]:
def model_fit(
    model, x_train, y_train, 
    val_rate = .2, batch_size = 1, epochs = 20,
    save = False, img_save = False, filename = 'model'
    ):
    result = model.fit(
    x_train, y_train,
        validation_split = val_rate,
        batch_size = batch_size,
        epochs = epochs
    )

    hist_dict = result.history

    loss = hist_dict['loss']
    val_loss = hist_dict['val_loss']

    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, 'b', label = 'Training loss')
    plt.plot(epochs, val_loss, 'g', label = 'Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('loss')
    plt.legend()

    if save == True:
        model.save(f'data/{filename}_fitted.h5')
        print('모델 저장 완료')
    
    if img_save == True:
        plt.savefig(f'data/{filename}.png')
    return result